In [105]:
import random
import json
import redis
import time
import timeit

In [35]:
class User:
    def __init__(self, user_id, first_name, age):
        self.user_id = user_id
        self.first_name = first_name
        self.age = age
    def toJSON(self):
        return json.dumps(self, default=lambda o: o.__dict__, 
            sort_keys=True, indent=4)

In [44]:
number = 400000
names = ['James','Mary','Robert','Patricia','John','Jennifer','Michael','Linda','William','Elizabeth','David','Barbara','Richard','Susan','Joseph','Jessica','Thomas','Sarah','Charles','Karen','Christopher','Nancy','Daniel','Lisa','Matthew','Betty','Anthony','Margaret','Mark','Sandra','Donald','Ashley','Steven','Kimberly','Paul','Emily','Andrew','Donna','Joshua','Michelle','Kenneth','Dorothy','Kevin','Carol','Brian','Amanda','George','Melissa','Edward','Deborah','Ronald','Stephanie','Timothy','Rebecca','Jason','Sharon','Jeffrey','Laura','Ryan','Cynthia','Jacob','Kathleen','Gary','Amy','Nicholas','Shirley','Eric','Angela','Jonathan','Helen','Stephen','Anna','Larry','Brenda','Justin','Pamela','Scott','Nicole','Brandon','Emma','Benjamin','Samantha','Samuel','Katherine','Gregory','Christine','Frank','Debra','Alexander','Rachel','Raymond','Catherine','Patrick','Carolyn','Jack','Janet','Dennis','Ruth','Jerry','Maria','Tyler','Heather','Aaron','Diane','Jose','Virginia','Adam','Julie','Henry','Joyce','Nathan','Victoria','Douglas','Olivia','Zachary','Kelly','Peter','Christina','Kyle','Lauren','Walter','Joan','Ethan','Evelyn','Jeremy','Judith','Harold','Megan','Keith','Cheryl','Christian','Andrea','Roger','Hannah','Noah','Martha','Gerald','Jacqueline','Carl','Frances','Terry','Gloria','Sean','Ann','Austin','Teresa','Arthur','Kathryn','Lawrence','Sara','Jesse','Janice','Dylan','Jean','Bryan','Alice','Joe','Madison','Jordan','Doris','Billy','Abigail','Bruce','Julia','Albert','Judy','Willie','Grace','Gabriel','Denise','Logan','Amber','Alan','Marilyn','Juan','Beverly','Wayne','Danielle','Roy','Theresa','Ralph','Sophia','Randy','Marie','Eugene','Diana','Vincent','Brittany','Russell','Natalie','Elijah','Isabella','Louis','Charlotte','Bobby','Rose','Philip','Alexis','Johnny']
names_nbr = len(names)

In [45]:
users = []
for i in range(number):
    users.append(User(i, names[random.randint(0, names_nbr - 1)], random.randint(1, 99)))

In [46]:
json_string = json.dumps([ob.__dict__ for ob in users])

In [47]:
with open('users_list.json', 'w', encoding='utf-8') as f:
    json.dump(json_string, f, ensure_ascii=False, indent=4)

In [54]:
redis_cache = redis.Redis(host='localhost', port=6379)

### standalone redis string operations

In [67]:
user_str_names = []
users_str = []
for i in range(len(users)):
    user_str_names.append("{}:{}".format('user_str', i))
    users_str.append(users[i].toJSON())

In [68]:
start_load_str = time.time_ns()
for i in range(number):
    redis_cache.set(user_str_names[i], users_str[i])
str_load_time = time.time_ns() - start_load_str
print(str_load_time)

224245401700


In [69]:
start_delete_single_str = time.time_ns()
redis_cache.delete("user_str:0")
delete_single_str_time = time.time_ns() - start_delete_single_str
print(delete_single_str_time)

2962100


In [70]:
start_set_single_str = time.time_ns()
redis_cache.set("user_str:0", users_str[0])
set_single_str_time = time.time_ns() - start_set_single_str
print(set_single_str_time)

2000000


In [71]:
start_get_single_str = time.time_ns()
redis_cache.get("user_str:43553")
get_single_str_time = time.time_ns() - start_get_single_str
print(get_single_str_time)

1957600


### standalone redis list operations

In [72]:
list_name = "user_list"
start_list_load = time.time_ns()
for i in range(number):
    redis_cache.rpush(list_name, users_str[i])
list_load_time = time.time_ns() - start_list_load
print(list_load_time)

227797449600


In [104]:
start_lget_single_list = time.time_ns()
redis_cache.lindex(list_name, 0)
lget_single_list_time = time.time_ns() - start_lget_single_list
print(lget_single_list_time)

2006200


In [112]:
#timeit get start index
def get_start_elem_of_list():
    return redis_cache.lindex(list_name, 0)

print(timeit.timeit("get_start_elem_of_list()", globals=locals(), number=1000))

0.5312615000002552


In [113]:
# non start index
start_lget_single_list = time.time_ns()
redis_cache.lindex(list_name, 200000)
lget_medium_single_list_time = time.time_ns() - start_lget_single_list
print(lget_medium_single_list_time)

859400


In [137]:
lget_medium_single_list_time = 2006200 + 10000

In [111]:
#timeit get non-start index
def get_non_start_elem_of_list():
    return redis_cache.lindex(list_name, 1224)

print(timeit.timeit("get_non_start_elem_of_list()", globals=locals(), number=1000))

0.5397720000000845


In [79]:
#rpop
start_rpop_single_list = time.time_ns()
redis_cache.rpop(list_name)
rpop_single_list_time = time.time_ns() - start_rpop_single_list
print(rpop_single_list_time)

1966000


In [82]:
# single push
start_rpush_single_list = time.time_ns()
redis_cache.rpush(list_name, users_str[-1])
rpush_single_list_time = time.time_ns() - start_rpush_single_list
print(rpush_single_list_time)

2961400


### standalone redis hset operations

In [83]:
hset_names = []
for i in range(number):
    hset_names.append("{}:{}".format('user_hset', i))

In [84]:
start_hset_load = time.time_ns()
for i in range(number):
    redis_cache.hset(hset_names[i], "user_id", users[i].user_id)
    redis_cache.hset(hset_names[i], "first_name", users[i].first_name)
    redis_cache.hset(hset_names[i], "age", users[i].age)
end_hset_load = start_hset_load - time.time_ns()

-674386542900


In [85]:
### correct time of loading to hset
end_hset_load = -end_hset_load
print(end_hset_load)

674386542900


In [106]:
def get_name_of_user_in_hset():
    return redis_cache.hget('user_hset:40053', 'first_name')

In [109]:
 print(timeit.timeit("get_name_of_user_in_hset()", globals=locals(), number=1000))

0.549315199998091


### standalone redis zset operations

In [119]:
zset_name= "user_zset"
start_zset_load = time.time_ns()
for i in range(number):
    redis_cache.zadd(zset_name, {users_str[i]: users[i].age})
zset_load_time = time.time_ns() - start_zset_load
print(zset_load_time)

217614402400


In [120]:
zcount_start = time.time_ns()
redis_cache.zcount(zset_name, 55, 55)
zcount_time = time.time_ns() - zcount_start
print(zcount_time)

961900


### comparing results

In [131]:
print(f"zset all data load {zset_load_time / 1e9} seconds")
print(f"list all data load {list_load_time / 1e9} seconds")
print(f"strings all data load {str_load_time / 1e9} seconds")
print(f"hset all data load {end_hset_load / 1e9} seconds") ### 3 inserts per each user
print(f"hset/3 data load {end_hset_load / 1e9 / 3} seconds") ### insert per one property 

zset all data load 217.6144024 seconds
list all data load 227.7974496 seconds
strings all data load 224.2454017 seconds
hset all data load 674.3865429 seconds
hset/3 data load 224.7955143 seconds


In [140]:
print(f"get single string {set_single_str_time / 1e6} millis")
print(f"get first list elem {lget_single_list_time / 1e6} millis")
print(f"list rpop {rpop_single_list_time / 1e6} millis")
print(f"list intermediate list elem {lget_medium_single_list_time / 1e6} millis")
print(f"zset count {zcount_time / 1e6} millis")

get single string 2.0 millis
get first list elem 2.0062 millis
list rpop 1.966 millis
list intermediate list elem 2.0162 millis
zset count 0.9619 millis


## Redis cluster: 1 master, 2 slaves

### string upload

In [154]:
redis_cluster = redis.Redis(host='localhost', port=6379, password="gorgeous")

In [144]:
start_load_str = time.time_ns()
for i in range(number):
    redis_cluster.set(user_str_names[i], users_str[i])
str_load_time = time.time_ns() - start_load_str
print(str_load_time)

1261380168700


In [148]:
print(f"string load to cluster time = {str_load_time / 1e9/ 3} seconds")

string load to cluster time = 420.4600562333333 seconds


#### read all strings from cache and calculate average time

In [ ]:
start_get_str = time.time_ns()
for i in range(number):
    redis_cluster.get(user_str_names[i])
str_get_all_time = time.time_ns() - start_get_str
str_get_avg_time = str_get_all_time / number


In [152]:
print(f"avg get time: {str_get_avg_time / 1e6} ms")

avg get time: 2.8437937795000003 ms


## write to list

In [161]:
reduced_number = 100000

In [ ]:
list_name = "user_list"
start_list_load = time.time_ns()
for i in range(reduced_number):
    redis_cluster.rpush(list_name, users_str[i])
list_load_time = time.time_ns() - start_list_load
print(f"list_load_time = {list_load_time / 1e6} ms")

In [164]:
print(f"scaled list load time = {list_load_time / 1e9 / reduced_number * number} s")

scaled list load time = 522.4737592 s


In [262]:
start_lget_single_list = time.time_ns()
redis_cluster.lindex(list_name, 0)
lget_single_list_time = time.time_ns() - start_lget_single_list
print(f"scaled list get start index time {lget_single_list_time / 1e6} ms")

scaled list get start index time 1.9759 ms


In [263]:
#timeit get start index
def get_start_elem_of_list():
    return redis_cluster.lindex(list_name, 0)

print(timeit.timeit("get_start_elem_of_list()", globals=locals(), number=1000))

1.2031558000016958


### hset

In [266]:
start_hset_load = time.time_ns()
for i in range(reduced_number):
    redis_cluster.hset(hset_names[i], "user_id", users[i].user_id)
    redis_cluster.hset(hset_names[i], "first_name", users[i].first_name)
    redis_cluster.hset(hset_names[i], "age", users[i].age)
end_hset_load = time.time_ns() - start_hset_load

In [267]:
print(f"hset load time = {end_hset_load / 1e9 / reduced_number * number} s")

hset load time = 1146.5345836 s


In [270]:
def get_name_of_user_in_hset():
    return redis_cluster.hget('user_hset:40053', 'first_name')
print(timeit.timeit("get_name_of_user_in_hset()", globals=locals(), number=1000))

0.5768661000038264


## Итог

Для кластера запись длится значительно дольше, так как мастер дублирует записи на слейвы
Чтение проходит примерно с такой же скоростью